In [4]:
# 데이터 가져오기
from google.colab import drive
import warnings
warnings.filterwarnings("ignore")

drive.mount('/content/gdrive/')
#https://arxiv.org/abs/2009.12042 => 하이퍼파라미터 최적화 논문

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
%tensorflow_version 1.5.0
import tensorflow as tf
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
pd.describe_option("display.max_columns",40)


path = "/content/gdrive/Shareddrives/DACON_fds/source/fds/"


train = pd.read_csv(path + "train.csv")
test = pd.read_csv(path +"test.csv")
val = pd.read_csv(path +"val.csv")
train.drop(columns={"ID"},inplace=True)
val.drop(columns={"ID"},inplace=True)

normal = val[val['Class']==0]
abnormal = val[val['Class']==1]

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.5.0`. This will be interpreted as: `1.x`.


After that, `%tensorflow_version 1.x` will throw an error.

Your notebook should be updated to use Tensorflow 2.
See the guide at https://www.tensorflow.org/guide/migrate#migrate-from-tensorflow-1x-to-tensorflow-2.

TensorFlow is already loaded. Please restart the runtime to change versions.


In [6]:
abnormal_index = [71,4039,9326,14221,28146]
random_predict = pd.read_csv("/content/gdrive/MyDrive/Dacon_신용카드이상탐지대회/random_predict_value.csv")
random_variable = pd.read_csv("/content/gdrive/MyDrive/Dacon_신용카드이상탐지대회/random_변수_score.csv")
random_predict.drop(columns="Unnamed: 0",inplace= True)
random_variable.drop(columns="Unnamed: 0",inplace= True)

In [8]:
abnormal.loc[abnormal_index]

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V22,V23,V24,V25,V26,V27,V28,V29,V30,Class
71,-3.043541,-3.157307,1.088463,2.288644,1.359805,-1.064823,0.325574,-0.067794,-0.270953,-0.838587,...,0.435477,1.375966,-0.293803,0.279798,-0.145362,-0.252773,0.035764,7.084469,-0.989438,1
4039,1.159373,2.844795,-4.050680,4.777701,2.948980,-2.010361,1.744086,-0.410287,-2.450198,-2.042168,...,-0.433470,-0.529323,-0.597020,1.335954,0.547092,0.009979,0.160769,-0.293440,-0.521811,1
9326,1.080433,0.962831,-0.278065,2.743318,0.412364,-0.320778,0.041290,0.176170,-0.966952,-0.194120,...,-0.057036,-0.053692,-0.026373,0.400300,0.072828,0.027043,0.063238,-0.307413,-0.236222,1
14221,-1.430864,-0.802529,1.123320,0.389760,-0.281214,-0.055123,1.326232,0.195700,-0.546890,-0.713474,...,0.014002,0.844946,0.114963,0.156365,-0.619437,-0.120351,0.035594,4.643750,0.001140,1
28146,1.991976,0.158476,-2.583441,0.408670,1.151147,-0.096695,0.223050,-0.068384,0.577829,-0.888722,...,-0.295135,-0.072173,-0.450261,0.313267,-0.289617,0.002988,-0.015309,0.286872,1.006309,1


In [27]:
pd.describe_option("display.max_columns",None)
random_predict.loc[abnormal_index].describe().loc['mean'].sort_values(ascending=False)

predict61     3.901188
predict144    1.927408
predict107    1.164427
predict23     1.144083
predict81     0.620086
                ...   
predict34    -9.070198
predict20    -9.109926
predict37    -9.217685
predict102   -9.742121
predict87    -9.830312
Name: mean, Length: 152, dtype: float64

In [31]:
random_predict.loc[abnormal_index]['predict61']

71       -3.238386
4039     47.082893
9326     -8.757102
14221    -4.958043
28146   -10.623421
Name: predict61, dtype: float64

In [55]:
target = val.loc[abnormal_index]
col = target.columns
for idx,value in enumerate(val.describe().loc['75%']):
  target[col[idx]] = target[col[idx]] - value

In [50]:
all_target = abnormal
col = all_target.columns
for idx,value in enumerate(val.describe().loc['75%']):
  all_target[col[idx]] = all_target[col[idx]] - value

In [65]:
first = all_target.max().loc[all_target.max().index.difference(['V24','V28','V30','V9','V10','V24',"Class"])].sort_values().head(10).index
second = target.max().loc[target.max().index.difference(['V24','V28','V30','V9','V10','V24',"Class"])].sort_values().head(10).index
not_meaningful_col = ['V24','V28','V30','V9','V10','V24']
not_meaningful_col = not_meaningful_col + list(first.intersection(second))
not_meaningful_col

In [72]:
random_variable.sort_values("score",inplace=True,ascending=False)

In [80]:
random_variable.iloc[0]['variable']

"['V19', 'V23', 'V2', 'V16', 'V21', 'V15', 'V18', 'V13', 'V1', 'V22', 'V24', 'V29', 'V11', 'V4', 'V30', 'V17', 'V14', 'V6', 'V28', 'V10']"

In [82]:
not_meaningful_col

['V24', 'V28', 'V30', 'V9', 'V10', 'V24', 'V3', 'V12', 'V14', 'V6', 'V26']

In [90]:
target.describe() 

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V22,V23,V24,V25,V26,V27,V28,V29,V30,Class
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.0
mean,-1.364103,-0.802210,-1.951643,1.382575,0.519505,-1.086822,0.165222,-0.358755,-1.327741,-1.388177,...,-0.595396,0.167051,-0.691901,0.144206,-0.327950,-0.155723,-0.024811,1.504569,-0.789832,1.0
std,2.108209,2.216847,2.301485,1.830485,1.211175,0.832667,0.754772,0.245380,1.114662,0.677354,...,0.333725,0.775434,0.293961,0.477012,0.434617,0.119400,0.065076,3.389692,0.743338,0.0
min,-4.359119,-3.960770,-5.062242,-0.349284,-0.879925,-2.387627,-0.525535,-0.734123,-3.046506,-2.494931,...,-0.961633,-0.675418,-1.038421,-0.196566,-0.860488,-0.341873,-0.096131,-1.085691,-1.631266,1.0
25%,-2.746442,-1.605991,-3.595003,-0.330374,-0.186348,-1.442088,-0.343774,-0.392220,-1.563260,-1.341485,...,-0.823299,-0.218267,-0.891663,-0.073132,-0.530667,-0.209451,-0.045228,-1.071718,-1.163639,1.0
50%,-0.235145,-0.644987,-1.289628,1.549600,0.552435,-0.698044,-0.241250,-0.391630,-1.143198,-1.291350,...,-0.585200,-0.199786,-0.735205,-0.039664,-0.386412,-0.086112,-0.045058,-0.491406,-0.878050,1.0
75%,-0.156205,0.159368,0.076900,2.004274,0.761093,-0.473960,0.759408,-0.147666,-0.867261,-1.166237,...,-0.514162,0.698852,-0.467775,0.047369,-0.168223,-0.079121,-0.017585,3.865472,-0.640688,1.0
max,0.676398,2.041332,0.111758,4.038657,2.350268,-0.432389,1.177261,-0.128137,-0.018479,-0.646884,...,-0.092686,1.229871,-0.326439,0.983024,0.306041,-0.062057,0.079947,6.306190,0.364481,1.0


In [91]:
 all_target.describe()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V22,V23,V24,V25,V26,V27,V28,V29,V30,Class
count,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,...,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.0
mean,-9.121388,3.179643,-11.825547,3.980472,-5.815286,-2.589263,-8.832950,0.664478,-4.510870,-8.480631,...,-0.677689,-0.199907,-1.069570,-0.761530,-0.401421,-0.702129,-0.331987,-0.814085,-1.343779,1.0
std,7.998185,4.424168,8.966188,3.457016,6.382148,1.470455,8.972103,4.886456,2.628173,6.296736,...,1.050613,0.817867,0.515176,0.769739,0.573921,2.016113,0.605328,1.937880,0.631028,0.0
min,-29.088902,-4.764233,-32.200443,-1.953331,-19.090023,-5.895531,-32.330978,-12.567312,-10.655189,-23.092615,...,-2.307465,-1.406881,-2.023239,-2.535030,-1.448666,-7.441682,-1.582888,-1.863970,-2.273094,1.0
25%,-15.464788,0.431065,-21.314515,0.885403,-11.344583,-3.253264,-15.196605,-0.561941,-5.685587,-13.886717,...,-1.345805,-0.788236,-1.446662,-1.029317,-0.734441,-1.676084,-0.795604,-1.849997,-1.843795,1.0
50%,-5.941108,1.947331,-7.775063,4.331674,-3.492593,-2.534899,-4.038309,0.010104,-3.838200,-5.940637,...,-0.834921,-0.368709,-1.033251,-0.649128,-0.467936,-0.127781,-0.125966,-1.797387,-1.332822,1.0
75%,-3.368132,5.706533,-5.171553,5.987692,-0.762397,-1.841747,-2.401578,1.900938,-2.609000,-3.871380,...,-0.186430,0.256980,-0.707806,-0.411447,-0.004750,0.370678,0.040494,-0.543597,-1.080029,1.0
max,-0.639181,14.890546,-0.899805,10.636584,1.751556,0.086246,0.610437,11.783468,-0.614787,-1.099647,...,2.404566,2.293024,-0.119295,0.768892,1.402639,1.944596,0.549962,5.527912,-0.277347,1.0
